In [2]:
import xarray as xr
import matplotlib.pyplot as plt
from fsspec_reference_maker.hdf import SingleHdf5ToZarr
from fsspec_reference_maker.combine import MultiZarrToZarr
import fsspec

In [3]:
fs = fsspec.filesystem('s3', anon=True)

In [8]:
flist = fs.glob('s3://noaa-goes17/ABI-L2-CMIPC/2021/050/*/*.nc')

In [11]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.39 GiB
Comm: tcp://127.0.0.1:36581,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.39 GiB
Comm: tcp://127.0.0.1:37945,Total threads: 2
Dashboard: http://127.0.0.1:39111/status,Memory: 3.85 GiB
Nanny: tcp://127.0.0.1:34605,


In [25]:
import dask
@dask.delayed
def gen_ref(f):
    
    with fs.open(f, 'rb') as infile:
        return SingleHdf5ToZarr(infile, f, inline_threshold=300).translate()

    

In [33]:
out = []
for f in flist[::60]:
    out.append(gen_ref(f))

In [38]:
dicts = [dask.compute(*out[:5])]

distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing


In [45]:
dicts = dicts[0]

In [48]:
mzz = MultiZarrToZarr(
    dicts,
    remote_protocol='s3',
    remote_options={'anon':True},
    xarray_concat_args={'dim' : 'time'}
)

In [49]:
mzz.translate()

/home/lsterzinger/anaconda3/lib/python3.8/site-packages/dask/array/core.py:4351: PerformanceWarning: Increasing number of chunks by factor of 12
  result = blockwise(
/home/lsterzinger/anaconda3/lib/python3.8/site-packages/dask/array/core.py:4351: PerformanceWarning: Increasing number of chunks by factor of 12
  result = blockwise(


ValueError: Zarr requires uniform chunk sizes except for final chunk. Variable named 'CMI' has incompatible dask chunks: ((1, 1), (375, 375, 1, 374, 375, 1, 374, 375, 1, 374, 375, 1, 374, 375, 1, 374, 375), (375, 375, 1, 374, 375, 1, 374, 375, 1, 374, 375, 1, 374, 375, 1, 374, 375, 1, 374, 375, 1, 374, 375, 1, 374, 375, 1, 374, 375)). Consider rechunking using `chunk()`.